In [166]:
!pip install anthropic rich pytz

In [167]:
from anthropic import Anthropic
from rich import print
from datetime import datetime, timezone, timedelta
import re, json, os, requests, pytz

In [168]:
os.environ["ANTHROPIC_API_KEY"] = "your_api_key"

In [169]:
client = Anthropic()
MODEL_NAME = "claude-3-opus-20240229"

road_condition_message = {
    "role": "user",
    "content": "What are the current road conditions from Hwy 16 to Maligne Canyon"
}

In [170]:
message = client.messages.create(
    model=MODEL_NAME,
    max_tokens=1024,
    messages=[road_condition_message]
).content[0].text

In [171]:
print("##### Response w/o Function Calling #####\n\n" + message)

##### Response w/o Function Calling #####

I apologize for the confusion, but I am not able to provide real-time road conditions as I am a language model AI 
and do not have access to live data. Road conditions can change rapidly based on weather, maintenance, and other 
factors.

For the most accurate and up-to-date information on road conditions from Highway 16 to Maligne Canyon, I recommend 
checking the following resources:

1. Alberta 511 (https://511.alberta.ca/): This official Alberta government website provides current road 
conditions, road reports, and interactive maps.

2. Parks Canada - Jasper National Park (https://www.pc.gc.ca/en/pn-np/ab/jasper): The official website for Jasper 
National Park offers information on road conditions and closures within the park, including the route to Maligne 
Canyon.

3. Local news and weather reports: Local media outlets often provide updates on road conditions, especially during 
inclement weather or emergencies.

Always exercise caution and adjust your driving based on the current conditions. If you are unsure about the safety
of driving, it is best to delay your trip until conditions improve.

In [172]:
def get_road_details(location_description):
    url = 'https://511.alberta.ca/api/v2/get/winterroads'
    response = requests.get(url)
    road_details = {}

    if response.status_code == 200:
        road_conditions = response.json()

        # Find the road condition entry that matches the given LocationDescription
        for road in road_conditions:
            if road['LocationDescription'] == location_description:
                # Convert Unix timestamp to datetime object in UTC
                last_updated_utc = datetime.fromtimestamp(int(road.get('LastUpdated', 0)), tz=timezone.utc)

                # Convert UTC datetime to Mountain Time
                mountain_time = pytz.timezone('America/Denver')
                last_updated_mountain = last_updated_utc.astimezone(mountain_time)

                # Format datetime object to a human-readable string
                last_updated_str = last_updated_mountain.strftime('%Y-%m-%d %H:%M:%S %Z')

                road_details = {
                    "Primary Condition": road.get('Primary Condition', 'N/A'),
                    "Secondary Conditions": road.get('Secondary Conditions', 'N/A'),
                    "Visibility": road.get('Visibility ', 'N/A'),
                    "Roadway Name": road.get('RoadwayName', 'N/A'),
                    "AreaName": road.get('AreaName', 'N/A'),
                    "Last Updated": last_updated_str
                }
                break  # Exit the loop once the matching road is found
    else:
        print("Failed to retrieve road conditions")

    return road_details

In [173]:
tool_description = """
<tool_description>
    <tool_name>get_road_details</tool_name>
    <description>
        Function for finding the current winter road conditions in Alberta.
    <parameters>
        <parameter>
            <name>location_description</name>
            <type>str</type>
            <description>LocationDescription of the road in Alberta</description>
        </parameter>
    </parameters>
</tool_description>
"""

In [174]:
system_prompt = f"""
In this environment you have access to a set of tools you can use to answer the
user's question.

You may call them like this:
<function_calls>
    <invoke>
        <tool_name>$TOOL_NAME</tool_name>
        <parameters>
            <$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
            ...
        </parameters>
    </invoke>
</function_calls>

Here are the tools available:
<tools>{tool_description}</tools>
"""

In [175]:
function_calling_message = client.messages.create(
    model=MODEL_NAME,
    max_tokens=1024,
    messages=[road_condition_message],
    system=system_prompt
).content[0].text

print(function_calling_message)

<function_calls>
    <invoke>
        <tool_name>get_road_details</tool_name>
        <parameters>
            <location_description>Hwy 16 to Maligne Canyon</location_description>
        </parameters>
    </invoke>
</function_calls>

In [176]:
def extract_between_tags(tag, string, strip=False):
    ext_list = re.findall(f"<{tag}>(.+?)</{tag}>", string, re.DOTALL)
    return [e.strip() for e in ext_list] if strip else ext_list

function_params = {"location_description": extract_between_tags("location_description", function_calling_message)[0]}
function_name = extract_between_tags("tool_name", function_calling_message)[0]
names_to_functions = {
    'get_road_details': get_road_details,
}
roadcond = names_to_functions[function_name](**function_params)

# Construct function results
function_results = f"""
<function_results>
  <result>
    <tool_name>get_road_details</tool_name>
    <stdout>{roadcond}</stdout>
  </result>
</function_results>"""

In [177]:
partial_assistant_message = function_calling_message + function_results

final_message = client.messages.create(
    model=MODEL_NAME,
    max_tokens=1024,
    messages=[
        road_condition_message,
        {
            "role": "assistant",
            "content": partial_assistant_message
        }
    ],
    system=system_prompt
).content[0].text

In [178]:
print("\n\n##### Response With Function Calling #####"+ final_message)

##### Response With Function Calling #####

According to the current road conditions report, the primary condition on the road from Highway 16 to Maligne 
Canyon is covered ice. Secondary conditions include shoulder ice/snow and packed snow and ice on the roadway. The 
visibility is not reported. This information is for the road named Maligne Road in the Jasper area, and was last 
updated on March 4, 2024 at 10:44 AM MST.